#### Import libraries

In [55]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

#### Import dataset

In [56]:
data = pd.read_csv('../house_prices_seattle_data.csv')
print(data.shape)

(21597, 21)


In [57]:
data.head(5)

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,7129300520,10/13/14,3,1.00,1180,5650,1.0,0,0,3,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,221900
1,6414100192,12/9/14,3,2.25,2570,7242,2.0,0,0,3,...,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,538000
2,5631500400,2/25/15,2,1.00,770,10000,1.0,0,0,3,...,770,0,1933,0,98028,47.7379,-122.233,2720,8062,180000
3,2487200875,12/9/14,4,3.00,1960,5000,1.0,0,0,5,...,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,604000
4,1954400510,2/18/15,3,2.00,1680,8080,1.0,0,0,3,...,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,510000


#### Check for data types

In [58]:
data.dtypes

id                 int64
date              object
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
price              int64
dtype: object

##### At a first glance, we can see that apart from the column date, the rest of the variables are treated as numerical data. First observation is that even though 'id' is considered as a numerical, it actually and identificator for each house(not for each sale, as will see later, some houses are duplicated, because they were sold in both of the years thatare in the dataset). We might also reconsider later on to treat some of the numerical variables as categorical ordinal(ex: bedrooms, view, zipcode)

#### Check for nan/null values

In [59]:
data.isna().sum()

id               0
date             0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
price            0
dtype: int64

#### Check for duplicates in the ID column to see if a property is more than one time in the dataset.

In [60]:
duplicates = data[data['id'].duplicated(keep=False) == True]
duplicates.shape

(353, 21)

##### We observe that some id's are repeated, and that the dates and prices are different. Therefore we can asume that the property was bougth and selled again between the two years represented in the dataframe. We proceed then to drop the rows where the data is earlier and keep the ones that were lastly bought.

In [61]:
data_nodup2 = data.sort_values('date').drop_duplicates('id',keep='last')
data_nodup = data_nodup2.sort_index(ignore_index=True)

#### Check for max and min values to check that no errors were done when creating dataset.

In [62]:
data_nodup.max()

id               9900000190
date                 9/9/14
bedrooms                 33
bathrooms                 8
sqft_living           13540
sqft_lot            1651359
floors                  3.5
waterfront                1
view                      4
condition                 5
grade                    13
sqft_above             9410
sqft_basement          4820
yr_built               2015
yr_renovated           2015
zipcode               98199
lat                 47.7776
long               -121.315
sqft_living15          6210
sqft_lot15           871200
price               7700000
dtype: object

In [63]:
data_nodup.min()

id               1000102
date             1/10/15
bedrooms               1
bathrooms            0.5
sqft_living          370
sqft_lot             520
floors                 1
waterfront             0
view                   0
condition              1
grade                  3
sqft_above           370
sqft_basement          0
yr_built            1900
yr_renovated           0
zipcode            98001
lat              47.1559
long            -122.519
sqft_living15        399
sqft_lot15           651
price              78000
dtype: object

##### The maximum value for bedrooms is 33, we'll check that case in order to see if it's a correct value or not.

In [64]:
data_nodup[data['bedrooms'] == 33]

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
15856,259601100,5/13/14,5,2.0,2290,7125,1.0,0,0,3,...,1190,1100,1964,0,98008,47.634,-122.119,1460,7920,580000


##### We see that this property has a total of 33 bedrooms in 1620 sqft_living, supposing that all rooms have the same size, each room will measure around 5 square metes. Also, there is a total of 1.75 bathrooms for the total of the property. Based on these considerations, we assume that there's a maybe a typo. We're removing the column anyway as it belongs to the range of prices in the majority class. 

In [65]:
data_nodup = data_nodup.drop(data_nodup[data_nodup['bedrooms'] == 33].index) 

#### Create a csv file with the cleaned data.

In [66]:
data_nodup.to_csv('data_clean.csv', index=False)